In [57]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from collections import Counter

In [58]:
df = pd.read_csv('tireoide.csv')
print(df)

   id sexo cirurgia  tsh    tt4 ti diagnostico
0  P1    F        N  6.8  156.2  N         Neg
1  P2    F        N  7.3  152.9  S         Neg
2  P3    M        S  8.8  148.4  N    HipoPrim
3  P4    F        N  6.9  132.7  N    HipoComp
4  P5    M        N  5.4  150.9  N         Neg


- **`Atributos qualitativos`**: sexo, cirurgia, ti, diagnostico
- **`Atributos quantitativos`**: id, tsh, tt4

In [59]:
# Codificando os atributos com map()
df['sexo'] = df['sexo'].map({'F': 0, 'M': 1})
df['cirurgia'] = df['cirurgia'].map({'N': 0, 'S': 1})
df['ti'] = df['ti'].map({'N': 0, 'S': 1})
df['diagnostico'] = df['diagnostico'].map({'Neg': 0, 'HipoPrim': 1, 'HipoComp': 2})
print(df)

   id  sexo  cirurgia  tsh    tt4  ti  diagnostico
0  P1     0         0  6.8  156.2   0            0
1  P2     0         0  7.3  152.9   1            0
2  P3     1         1  8.8  148.4   0            1
3  P4     0         0  6.9  132.7   0            2
4  P5     1         0  5.4  150.9   0            0


In [60]:
# Normalizando tsh e tt4
df['tsh_normalizado'] = (df['tsh'] - df['tsh'].min()) / (df['tsh'].max() - df['tsh'].min())
df['tt4_normalizado'] = (df['tt4'] - df['tt4'].min()) / (df['tt4'].max() - df['tt4'].min())

# Mostrando o conjunto de dados com os valores normalizados
print("\nConjunto de dados com tsh e tt4 normalizados:")
print(df[['id', 'tsh', 'tsh_normalizado', 'tt4', 'tt4_normalizado']])


Conjunto de dados com tsh e tt4 normalizados:
   id  tsh  tsh_normalizado    tt4  tt4_normalizado
0  P1  6.8         0.411765  156.2         1.000000
1  P2  7.3         0.558824  152.9         0.859574
2  P3  8.8         1.000000  148.4         0.668085
3  P4  6.9         0.441176  132.7         0.000000
4  P5  5.4         0.000000  150.9         0.774468


In [61]:
# Atributos do novo paciente: (F;N;7.0;150.0;S)
novo_paciente = {
    'sexo': 'F', 
    'cirurgia': 'N', 
    'tsh': 7.0, 
    'tt4': 150.0, 
    'ti': 'S'
}

print(novo_paciente)

{'sexo': 'F', 'cirurgia': 'N', 'tsh': 7.0, 'tt4': 150.0, 'ti': 'S'}


In [62]:
# Normalizando os atributos TSH e TT4 do novo paciente
novo_paciente['tsh_normalizado'] = (novo_paciente['tsh'] - df['tsh'].min()) / (df['tsh'].max() - df['tsh'].min())
novo_paciente['tt4_normalizado'] = (novo_paciente['tt4'] - df['tt4'].min()) / (df['tt4'].max() - df['tt4'].min())
print(novo_paciente)

# Função para calcular a distância Euclidiana
def calcular_distancia_euclidiana(paciente1, paciente2):
    distancia = euclidean([paciente1['sexo'], paciente1['cirurgia'], paciente1['tsh_normalizado'], paciente1['tt4_normalizado'], paciente1['ti']],
                          [paciente2['sexo'], paciente2['cirurgia'], paciente2['tsh_normalizado'], paciente2['tt4_normalizado'], paciente2['ti']])
    return distancia

print("Valores únicos em 'sexo':", df['sexo'].unique())
print("Valores únicos em 'cirurgia':", df['cirurgia'].unique())
print("Valores únicos em 'ti':", df['ti'].unique())


{'sexo': 'F', 'cirurgia': 'N', 'tsh': 7.0, 'tt4': 150.0, 'ti': 'S', 'tsh_normalizado': np.float64(0.4705882352941175), 'tt4_normalizado': np.float64(0.736170212765958)}
Valores únicos em 'sexo': [0 1]
Valores únicos em 'cirurgia': [0 1]
Valores únicos em 'ti': [0 1]


In [65]:
# Mapeia valores categóricos para numéricos (0 e 1), incluindo tratamento para valores desconhecidos
df['sexo'] = df['sexo'].map({'F': 0, 'M': 1}).fillna(-1).astype(int)  # Usando -1 para valores não mapeados
df['cirurgia'] = df['cirurgia'].map({'N': 0, 'S': 1}).fillna(-1).astype(int)
df['ti'] = df['ti'].map({'N': 0, 'S': 1}).fillna(-1).astype(int)

# Converta os dados categóricos do novo paciente para numéricos
novo_paciente['sexo'] = 0  # F -> 0
novo_paciente['cirurgia'] = 0  # N -> 0
novo_paciente['ti'] = 1  # S -> 1

print(df)

   id  sexo  cirurgia  tsh    tt4  ti  diagnostico  tsh_normalizado  \
0  P1    -1        -1  6.8  156.2  -1            0         0.411765   
1  P2    -1        -1  7.3  152.9  -1            0         0.558824   
2  P3    -1        -1  8.8  148.4  -1            1         1.000000   
3  P4    -1        -1  6.9  132.7  -1            2         0.441176   
4  P5    -1        -1  5.4  150.9  -1            0         0.000000   

   tt4_normalizado  
0         1.000000  
1         0.859574  
2         0.668085  
3         0.000000  
4         0.774468  


In [64]:
# Calculando as distâncias para cada paciente no conjunto de dados
distancias = []
for i, row in df.iterrows():
    distancia = calcular_distancia_euclidiana(novo_paciente, row)
    distancias.append((distancia, row['diagnostico']))

# Ordenando os pacientes pela distância Euclidiana
distancias_ordenadas = sorted(distancias, key=lambda x: x[0])

# Função para classificar o novo paciente com base em k vizinhos mais próximos
def classificar(k):
    k_vizinhos = distancias_ordenadas[:k]
    diagnosticos = [diagnostico for _, diagnostico in k_vizinhos]
    return Counter(diagnosticos).most_common(1)[0][0]

# Classificando para k = 1, k = 3 e k = 5
resultado_k1 = classificar(1)
resultado_k3 = classificar(3)
resultado_k5 = classificar(5)

print(f"Classificação para k=1: {resultado_k1}")
print(f"Classificação para k=3: {resultado_k3}")
print(f"Classificação para k=5: {resultado_k5}")

Classificação para k=1: 0
Classificação para k=3: 0
Classificação para k=5: 0
